# 01 - Preparation

In [2]:
import os
import warnings

import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
import polars as pl
import polars.selectors as cs
import tensorflow as tf
from plotly.subplots import make_subplots

print("TensorFlow version:", tf.version.VERSION)

plotly.offline.init_notebook_mode(connected=True)  # for nbviewer
plotly.io.templates.default = "plotly_dark"

TensorFlow version: 2.16.2


# Description of the dataset

In [22]:
# TODO explain the data here

In [23]:
TARGET_STEPS = 16  # length of the target sequences

FAMILY_COUNT = 33  # number of families in the dataset
STORE_COUNT = 54  # number of stores in the dataset

# Read the training and target dataframes

In [24]:
# NOTE We ignore the 'holidays_events.csv' dataframe

In [25]:
# NOTE The training dataframe contains 1684 unique dates
# NOTE from 2013-01-01 to 2017-08-31, missing all four Dec. 25.
# NOTE We ignore the missing Dec. 25

# NOTE The 'store_nbr' in the origninal dataframes are ordered wrong,
# NOTE they are ordered by first digit, i.e. 1, 10, 11, ..., 9,
# NOTE so we must be careful and preserve the order everywhere.

In [26]:
df = pl.read_csv(os.path.join("input", "train.csv"), try_parse_dates=True)
df = df.drop("id")  # the id does nothing for us

# read the target dataframe and add the missing 'sales' column
tail = (
    pl.read_csv(os.path.join("input", "test.csv"), try_parse_dates=True)
    .with_columns(sales=pl.lit(None))  # column with null values
    .select(df.columns)  # reorder the columns to match the training dataframe
)

# append the target to the training dataframe
df = pl.concat([df, tail])

# encode the 'family' as integers in an integer column 'family_nbr'
# maintain the order (alphabetical) to have the 'family_nbr' follow the same order
df = df.with_columns(
    pl.col("family")
    .cast(pl.Enum(df["family"].unique(maintain_order=True)))
    .to_physical()  # cast Enum values to u32 (start = 0)
    .alias("family_nbr")
    + 1  # start at 1 like the store_nbr
)

# Add covariates from other competition input files

In [27]:
# NOTE Add the oil prices to the dataframe
# NOTE We have to oil prices for the target time-steps

oil = pl.read_csv(os.path.join("input", "oil.csv"), try_parse_dates=True)
oil = oil.set_sorted("date").upsample("date", every="1d")  # add missing dates
oil = oil.interpolate().fill_null(strategy="backward")  # fill missing values

df = df.join(oil, on="date")

In [28]:
# NOTE Include the store type and the cluster from 'stores.csv'

stores = (
    pl.read_csv(os.path.join("input", "stores.csv"))
    .select("store_nbr", "type", "cluster")
    .rename({"type": "store_type", "cluster": "store_cluster"})
)

# encode the store type as integers
stores = stores.with_columns(
    pl.col("store_type")
    .cast(pl.Enum(stores["store_type"].unique(maintain_order=True)))
    .to_physical()
)

df = df.join(stores, on="store_nbr")

In [29]:
# NOTE Include the daily transactions per store from 'transactions.csv'
# NOTE This file is missing about 250k values (of about 3mil), which we will need to fill.
# NOTE It is questionable whether a column with this many missing values should be included.


# add the 'transactions' column to the dataframe
df = df.join(
    pl.read_csv(os.path.join("input", "transactions.csv"), try_parse_dates=True),
    on=["date", "store_nbr"],
    how="full",
    coalesce=True,
)

# fill missing values by interpolation (when possible) or backwards (not filling the target)
df = df.with_columns(pl.col("transactions").interpolate().backward_fill())

# Add relevant time signals

In [30]:
# NOTE We compute the DFT of the total sales to get an idea of what time signals are important.

# NOTE We aggregate with the idea that we can capture all relevant time signals from the total sales amount.
# NOTE By looking at the data, we can see that the time-periodicity of the 'sales' values depends greatly on the
# NOTE family, e.g. the family 'SCHOOL AND OFFICE SUPPLIES' exhibits a very strong yearly periodicity
# NOTE (with a peak in September and almost no sales the rest of the year), while other families like 'GROCERY I'
# NOTE show no yearly periodicity.
# NOTE By aggregating, we create a signal which contains all frequencies, more or less muted depending on the volume
# NOTE of sales of the exhibiting families (the values for 'GROCERY I' are much larger than those for 'SCHOOL ...').

In [31]:
# aggregate the 'sales' values to obtain the total sales per day
total_sales = (
    df.select("date", "sales")
    .group_by("date", maintain_order=True)
    .sum()
    .head(-TARGET_STEPS)
)

# compute the DFT of the sales and the corresponding frequencies
dft_df = (
    pl.DataFrame({"DFT": tf.abs(tf.signal.rfft(total_sales["sales"])).numpy()})
    .with_row_index()  # add index column
    .with_columns(  # scale the index to have the frequencies per year
        ((pl.col("index") * 365.25) / len(total_sales)).alias("freq")
    )
)

px.line(dft_df, x="freq", y="DFT").update_layout(
    height=350,
    xaxis=dict(title="frequency (year)"),
    yaxis=dict(visible=False, type="log"),
    title="log-DFT of total sales",
)

In [32]:
# NOTE We see the following spikes, in order of magnitude:
# NOTE - 0 -> trend
# NOTE - 52 -> period of one week
# NOTE - 104 -> period of a half-week
# NOTE - 24 -> period of one half-month
# NOTE - 1 -> period of one year
# NOTE - 12 -> period one month (small)

# NOTE We ignore the frequencies in (0, 1) as they correspond to periods greater than one year
# NOTE and our dataset does not contain enough years to make sense of these.

In [33]:
# NOTE Plot the weekly (and biweekly) and monthly periodic phenomena
# NOTE We see that the half-week element stems from the fact that the weekend differs
# NOTE from the rest of the week, giving a two-days block which repeats weekly.

# compute total sales per day of week and day of month
wd_sales = total_sales.group_by(pl.col("date").dt.weekday()).sum().sort(by="date")
md_sales = total_sales.group_by(pl.col("date").dt.day()).sum().sort(by="date")

fig = make_subplots(cols=2, subplot_titles=["day of week", "day of month"])
fig.add_trace(go.Bar(x=wd_sales["date"], y=wd_sales["sales"]), row=1, col=1)
fig.add_trace(go.Bar(x=md_sales["date"], y=md_sales["sales"]), row=1, col=2)
fig.update_traces(marker=dict(color=px.colors.qualitative.Plotly[0]))
fig.update_layout(
    height=350,
    showlegend=False,
    title="Total sales per day of week and day of month",
).update_xaxes(visible=False).update_yaxes(visible=False)

In [34]:
# NOTE We add signals corresponding to the frequencies identified above, that is:
# NOTE half-week, week, month, year
# NOTE Only use a monthly signal to capture the bimonthly periodicity

# add a timestamp column (one step per day)
df = df.with_columns(pl.col("date").cast(pl.Int32).alias("ts"))

df = df.with_columns(
    (2 * np.pi * pl.col("ts") / 3.5).sin().alias("sin_hwk"),  # half-week
    (2 * np.pi * pl.col("ts") / 3.5).cos().alias("cos_hwk"),
    (2 * np.pi * pl.col("ts") / 7).sin().alias("sin_wk"),  # week
    (2 * np.pi * pl.col("ts") / 7).cos().alias("cos_wk"),
    (2 * np.pi * pl.col("ts") / 30.5).sin().alias("sin_mth"),  # month
    (2 * np.pi * pl.col("ts") / 30.5).cos().alias("cos_mth"),
    (2 * np.pi * pl.col("ts") / 365.25).sin().alias("sin_yr"),  # year
    (2 * np.pi * pl.col("ts") / 365.25).cos().alias("cos_yr"),
)

# drop timestamp column as we prefer to keep the 'date' column in the dataframe
df = df.drop("ts")

# Correlation analysis

In [35]:
# NOTE Reorder the columns to have the temporal features first.
# NOTE Start with 'date', then 'sales' (target var) followed by 'transactions' as these
# NOTE are the variables for which we do not have values on the target set.
# NOTE The 4 non-temporal categorical features (depending only on the key) are at the end.

df = df.select(
    "date",
    "sales",
    "transactions",
    "onpromotion",
    "dcoilwtico",
    cs.contains("sin", "cos"),  # time signals
    cs.contains("store", "family"),  # time-independent (key)
)

In [36]:
# NOTE Look at correlation between columns on a per-key basis, as looking at correlation
# NOTE on `df` as a whole is not meaningful since the 'sales' values for different keys
# NOTE are correlated differently to the covariates

# NOTE We include the 'date' column (which gets cast to a daily timestamp as above) as is captures
# NOTE the overall increasing trend of the series, and is therefore a meaningful covariate.

# NOTE We get a warning when computing correlations as some variables are constant
# NOTE on some fibers of the key axis (notably, 'sales' is uniformly zero for some keys).
# NOTE We set the missing correlation values to zero and disable the warning.

# NOTE We observe that all covariates are correlated to the 'sales' values of at least some of the keys.

In [37]:
# partition the dataframe by key (without target time-steps)
kdfs = df.head(-TARGET_STEPS * STORE_COUNT * FAMILY_COUNT).partition_by(
    ["store_nbr", "family_nbr"],
    include_key=False,
    as_dict=True,
    maintain_order=True,
)

# compute correlations to 'sales' for each key (drop the categorical columns)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")  # disable the warning
    corr = [
        kdf.drop(cs.contains("store", "family")).corr()[df.columns.index("sales")]
        for kdf in kdfs.values()
    ]


# concatenate into a dataframe with rows = keys, cols = vars
corr = pl.concat(corr).drop("sales").fill_nan(0)

px.box(corr).update_layout(
    title="Correlation of 'sales' to the covariates (per key)",
    yaxis=dict(title="corr"),
    height=350,
)

In [38]:
# NOTE Compute autocorrelation for the 'sales' column (per key)
# NOTE We see quite a strong temporal correlation, which diminishes as the time gap increases.

# pivot to create a dataframe containing the 'sales' values for each key
key_sales = (
    df.select("date", "sales", "store_nbr", "family_nbr")
    .pivot(index="date", on=["store_nbr", "family_nbr"], maintain_order=True)
    .head(-TARGET_STEPS)
    .drop("date")
)

# compute the autocorrelation of the key 'sales' values for various shifts
autocorr = {
    f"{shift}": [
        key_sales.select(pl.corr(key, pl.col(key).shift(shift))).item()
        for key in key_sales.columns
    ]
    for shift in [1, 7, 14, 30, 60, 90, 120]
}

px.box(pl.DataFrame(autocorr)).update_layout(
    title="autocorrelation of 'sales' (per key)",
    xaxis=dict(title="shift (time-steps)"),
    yaxis=dict(title="corr"),
    height=350,
)

# Save the dataframe to file

In [39]:
# NOTE Save the dataframe to the input folder to use in other notebooks

df.write_csv(os.path.join("input", "df.csv"))